In [2]:
! pip install -q git+https://github.com/qubvel/segmentation_models.pytorch

^C


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\ARYAN\\anaconda3\\Lib\\site-packages\\torch\\utils\\__init__.py'
Consider using the `--user` option or check the permissions.



In [ ]:
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

import pandas as pd

In [ ]:
data_base = Path("/kaggle/input/paip-2019/PAIP_2019/PAIP_2019/")

In [ ]:
df = pd.read_csv(Path("../input/paip-2019/training.csv"))

In [ ]:
IDX = 1000

patch_path = Path(data_base, df.iloc[IDX]['patch'])
patch = Image.open(patch_path)
seg_path = Path(data_base, df.iloc[IDX]['seg'])
seg_img = Image.open(seg_path)
    
fig, ax = plt.subplots(1, 2, figsize=(8,8))
for a, img in zip(ax, [patch, seg_img]):
    a.imshow(img)
    a.set_xticks([])
    a.set_yticks([])

plt.tight_layout()
plt.show()

In [ ]:
import torch
import torch.utils.data as data

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.device_count())

In [ ]:
# Work on albumentations for the next iteration

from typing import Tuple
import numpy as np
from PIL import Image
import torchvision.transforms.functional as F

class WSIDataset(data.Dataset):
    def __init__(self, paths: pd.DataFrame, preprocess, size: Tuple[int, int] = (256, 256)):
        self.paths = paths
        self.size = size
        self.preprocess = preprocess
        
    def __len__(self):
        return len(self.paths.index)
    
    def __getitem__(self, idx):
        cur_path = self.paths.iloc[idx]
        patch_file = Path(data_base,cur_path['patch'])
        seg_file = Path(data_base,cur_path['seg'])
        
        patch = Image.open(patch_file)
        patch = np.array(patch.resize(self.size))
        # preprocess
        patch = self.preprocess(patch)
        patch = torch.from_numpy(patch.transpose(2, 0, 1))
        
        mask = Image.open(seg_file)
        mask = np.array(mask.resize(self.size))
        mask = mask[:, :, np.newaxis]
        mask = torch.from_numpy(mask.transpose(2, 0, 1))
        
        return patch.float(), mask.float()

In [ ]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="efficientnet-b0", 
    activation='sigmoid',
    encoder_weights = 'imagenet',
    in_channels=3,                  
    classes=1,                      
)
model = model.to(device, memory_format=torch.channels_last).train()

In [ ]:
preprocess = smp.encoders.get_preprocessing_fn('efficientnet-b0', pretrained='imagenet')
dataset = WSIDataset(df, preprocess, (224, 224))
train_data_loader = data.DataLoader(dataset, batch_size=96, shuffle=True, pin_memory=True, num_workers = 2)

In [ ]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

optimiser = optim.Adam(model.parameters(), 0.0005)
loss = smp.losses.DiceLoss("binary")
scheduler = lr_scheduler.LambdaLR(optimiser, lr_lambda=lambda epoch: 0.8 ** epoch)

In [ ]:
import gc
from tqdm import tqdm
import numpy as np

# next iter
PATH = Path("./unet")
PATH.mkdir(exist_ok=True)
MODEL_PATH = Path(PATH, "model.pt")
MODEL_STATE = Path(PATH, "model_state.pt")
MODEL_HIST = Path(PATH, "hist.npy")
OPT_STATE = Path(PATH, "optimiser.pt")

#prev iter
PREV = Path("../input/patch-level-segmentation-unet-2/unet")
PREV_MODEL_PATH = Path(PREV, "model.pt")
PREV_MODEL_STATE = Path(PREV, "model_state.pt")
PREV_MODEL_HIST = Path(PREV, "hist.npy")
PREV_OPT_STATE = Path(PREV, "optimiser.pt")
state = []

if PREV_MODEL_STATE.is_file():
    print("Loading previous model state....")
    model.load_state_dict(torch.load(str(PREV_MODEL_STATE),map_location=device))

if PREV_MODEL_HIST.is_file():
    print("Loading previous training history....")
    state=list(np.load(PREV_MODEL_HIST))
    
# if PREV_OPT_STATE.is_file():
#     print("Loading previous optimiser state...")
#     optimiser.load_state_dict(torch.load(str(PREV_OPT_STATE), map_location=device))
    
print(optimiser.state_dict)

In [ ]:
epochs = 4

for epoch in range(epochs):
    epoch_loss = 0
    epoch_iou = 0
    with tqdm(train_data_loader, unit="batch") as tepoch:
        for cc, (i, t) in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch}")
            i, t = i.to(device,memory_format=torch.channels_last), t.to(device, memory_format=torch.channels_last)
            optimiser.zero_grad()
            outputs = model(i)
            ls = loss(outputs, t)
            ls.backward()
            optimiser.step()
            with torch.no_grad():
                pred = (outputs > 0.5).float()
                tp, fp, fn, tn = smp.metrics.get_stats(pred.long(), t.long(), mode="binary")
                iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
                epoch_loss += ls.item()
                epoch_iou += iou.item()
                torch.cuda.empty_cache()
                lr = scheduler.get_last_lr()
                tepoch.set_postfix(loss=epoch_loss/(cc + 1), iou = epoch_iou/(cc + 1), lr=lr)
    scheduler.step()

    torch.save(model, str(MODEL_PATH))
    torch.save(model.state_dict(), str(MODEL_STATE))
    torch.save(optimiser.state_dict(), str(OPT_STATE))
    state.append((epoch_loss / len(dataset), epoch_iou / len(dataset)))
    gc.collect()
np.save(MODEL_HIST, state)